In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from tqdm import tqdm

In [2]:
# Гиперпараметры
# input_size = 3 * 455 * 500  # ???
num_classes = 211
learning_rate = 0.001
batch_size = 100
num_epochs = 5

In [3]:
new_size = (182, 200)
# Загрузка и предобработка датасета Country211
transform = transforms.Compose([
    transforms.Resize(size=new_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
# train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
# test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
train_dataset = torchvision.datasets.Country211(root='./data', split='train', transform=transform, download=True)
test_dataset = torchvision.datasets.Country211(root='./data', split='test', transform=transform, download=True)

In [5]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
# Определение модели CNN
class CNN(nn.Module):  # Здесь мы создаем класс CNN, который наследуется от nn.Module. Этот класс будет представлять нашу сверточную нейронную сеть.
    def __init__(self):
        super(CNN, self).__init__()
        # Здесь мы создаем последовательность слоев (nn.Sequential), которая будет представлять первый слой нашей модели. 
        # В этой последовательности мы добавляем следующие слои:
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=2, padding=2),  # Сверточный слой с 3 входными каналами (RGB), 192 выходными каналами и ядром размером 8x8. 
            nn.BatchNorm2d(32),  # Нормализация по батчу для 192 выходных каналов.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)  # Слой пулинга (максимальное значение) с ядром размером 2x2 и шагом 2.
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=4, padding=2),  # Сверточный слой с 3 входными каналами (RGB), 192 выходными каналами и ядром размером 8x8. 
            nn.BatchNorm2d(32),  # Нормализация по батчу для 192 выходных каналов.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)  # Слой пулинга (максимальное значение) с ядром размером 2x2 и шагом 2.
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=6, padding=2),  # Сверточный слой с 3 входными каналами (RGB), 192 выходными каналами и ядром размером 8x8. 
            nn.BatchNorm2d(32),  # Нормализация по батчу для 192 выходных каналов.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)  # Слой пулинга (максимальное значение) с ядром размером 2x2 и шагом 2.
        )

        self.fc1 = nn.Linear(16896, 512)  #  Здесь мы создаем полносвязный слой (nn.Linear) с 8192 входами и 512 выходами.
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 211)  # Еще один полносвязный слой с 512 входами и 10 выходами. Обычно это количество классов в задаче классификации.
    
    def forward(self, x):  # Метод forward определяет, как данные будут передаваться через модель.
        out = self.layer1(x)  # Пропускаем входные данные через первый слой.
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)  # Разворачиваем тензор, чтобы подготовить его для полносвязных слоев.
        out = self.fc1(out)  # Пропускаем через первый полносвязный слой.
        out = self.fc2(out)  # Пропускаем через второй полносвязный слой.
        out = self.fc3(out)
        return out

In [10]:
# Инициализация модели, функции потерь и оптимизатора
model = CNN()

# Эта строка создает экземпляр функции потерь CrossEntropyLoss. Эта функция используется для вычисления ошибки между предсказаниями модели и истинными
criterion = nn.CrossEntropyLoss()

# Здесь мы создаем оптимизатор Adam. Он обновляет параметры модели на основе градиентов, вычисленных во время обратного распространения ошибки. 
# Параметр lr устанавливает скорость обучения (learning rate), которая контролирует, насколько быстро модель адаптируется к данным.
optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

In [11]:
# Обучение модели
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [100/317], Loss: 5.6908
Epoch [1/5], Step [200/317], Loss: 5.4003
Epoch [1/5], Step [300/317], Loss: 5.3653
Epoch [2/5], Step [100/317], Loss: 5.2732
Epoch [2/5], Step [200/317], Loss: 5.2049
Epoch [2/5], Step [300/317], Loss: 5.1582
Epoch [3/5], Step [100/317], Loss: 5.1183
Epoch [3/5], Step [200/317], Loss: 5.0179
Epoch [3/5], Step [300/317], Loss: 5.2002
Epoch [4/5], Step [100/317], Loss: 5.0245
Epoch [4/5], Step [200/317], Loss: 5.0090
Epoch [4/5], Step [300/317], Loss: 5.1287
Epoch [5/5], Step [100/317], Loss: 5.0628
Epoch [5/5], Step [200/317], Loss: 5.1473
Epoch [5/5], Step [300/317], Loss: 4.8622


In [12]:
# Оценка модели
model.eval()  # Переключение в режим оценки
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total}%')

Accuracy of the model on the 10000 test images: 2.9668246445497632%
